<a href="https://colab.research.google.com/github/rohanjaiswal2001/ISL-Player-Stats/blob/main/ISL_Player_Stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import numpy as np
import pandas as pd
from copy import deepcopy
from google.colab import files

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
uploaded = files.upload()

Saving isl_player_final(1).csv to isl_player_final(1).csv


In [ ]:
isl_df = pd.read_csv(io.BytesIO(uploaded['isl_player_final(1).csv']))
isl_df

In [ ]:
isl_df.info()

In [ ]:
isl_df.describe().round(1)

Position Id's
1. - Defender
2. - Forward
3. - Midfielder

In [7]:
isl_df[(isl_df["minutes_played"] > 1500) & (isl_df["country_id"] == 1)].shape

(49, 93)

In [ ]:
isl_df[(isl_df["country_id"] == 1) & (isl_df["position_id"] == 2)]

Creating Variables for Players of different positions

In [19]:
indian_forwards = deepcopy(isl_df[(isl_df["country_id"] == 1) & (isl_df["position_id"] == 2)])

In [20]:
indian_midfielders = deepcopy(isl_df[(isl_df["country_id"] == 1) & (isl_df["position_id"] == 3)])


In [21]:
indian_defenders = deepcopy(isl_df[(isl_df["country_id"] == 1) & (isl_df["position_id"] == 1)])


In [22]:
indian_forwards.reset_index(drop=True, inplace=True)
indian_midfielders.reset_index(drop=True, inplace=True)
indian_defenders.reset_index(drop=True, inplace=True)

Leveling and Scaling for Per90 Stats

In [28]:
#Minutes played by players
fwd_mins_df = indian_forwards.groupby(["id", "name"])["minutes_played"].sum().reset_index()
md_mins_df = indian_midfielders.groupby(["id", "name"])["minutes_played"].sum().reset_index()
def_mins_df = indian_defenders.groupby(["id", "name"])["minutes_played"].sum().reset_index()

In [29]:
#Plotting minutes played by players on a Bar Graph
px.bar(x="name", y="minutes_played", text="minutes_played", data_frame=fwd_mins_df)
px.bar(x="name", y="minutes_played", text="minutes_played", data_frame=md_mins_df)
px.bar(x="name", y="minutes_played", text="minutes_played", data_frame=def_mins_df)

In [30]:
##Average Staitistics Per 90 Minutes for players
indian_forwards["goalsPer90"] = indian_forwards["events.goals"].divide(indian_forwards["minutes_played"]).multiply(90)
indian_midfielders["goalsPer90"] = indian_midfielders["events.goals"].divide(indian_midfielders["minutes_played"]).multiply(90)
indian_defenders["goalsPer90"] = indian_defenders["events.goals"].divide(indian_defenders["minutes_played"]).multiply(90)

In [31]:
indian_forwards = indian_forwards[indian_forwards["minutes_played"] > 0]
indian_midfielders = indian_midfielders[indian_midfielders["minutes_played"] > 0]
indian_defenders = indian_defenders[indian_defenders["minutes_played"] > 0]

In [32]:
indian_forwards["assistsPer90"] = indian_forwards["events.assists"].divide(indian_forwards["minutes_played"]).multiply(90)
indian_forwards["shots_on_targetPer90"] = indian_forwards["events.shots_on_target"].divide(indian_forwards["minutes_played"]).multiply(90)

In [33]:
indian_midfielders["assistsPer90"] = indian_midfielders["events.assists"].divide(indian_midfielders["minutes_played"]).multiply(90)
indian_midfielders["shots_on_targetPer90"] = indian_midfielders["events.shots_on_target"].divide(indian_midfielders["minutes_played"]).multiply(90)

In [34]:
indian_defenders["assistsPer90"] = indian_defenders["events.assists"].divide(indian_defenders["minutes_played"]).multiply(90)
indian_defenders["shots_on_targetPer90"] = indian_defenders["events.shots_on_target"].divide(indian_defenders["minutes_played"]).multiply(90)

In [36]:
per90Cols = ["events.goals", "events.assists", "events.shots", "events.shots_on_target", "events.chances_created",  
             "events.fouls_suffered", "touches.total", 'touches.aerial_duel.won']

In [37]:
for col in per90Cols:
    indian_forwards[col + "Per90"] = indian_forwards[col].divide(indian_forwards["minutes_played"]).multiply(90)
    indian_midfielders[col + "Per90"] = indian_midfielders[col].divide(indian_midfielders["minutes_played"]).multiply(90)
    indian_defenders[col + "Per90"] = indian_defenders[col].divide(indian_defenders["minutes_played"]).multiply(90)

In [38]:
cols_for_radar = []
for i in per90Cols:
    cols_for_radar.append(i + "Per90")
print(cols_for_radar)

['events.goalsPer90', 'events.assistsPer90', 'events.shotsPer90', 'events.shots_on_targetPer90', 'events.chances_createdPer90', 'events.fouls_sufferedPer90', 'touches.totalPer90', 'touches.aerial_duel.wonPer90']


In [40]:
[i + "Per90" for i in per90Cols]

['events.goalsPer90',
 'events.assistsPer90',
 'events.shotsPer90',
 'events.shots_on_targetPer90',
 'events.chances_createdPer90',
 'events.fouls_sufferedPer90',
 'touches.totalPer90',
 'touches.aerial_duel.wonPer90']

In [41]:
cols_for_radar = [i + "Per90" for i in per90Cols] #Initialising cols_for_radar to create variable for each of these statistics 
print(cols_for_radar)

['events.goalsPer90', 'events.assistsPer90', 'events.shotsPer90', 'events.shots_on_targetPer90', 'events.chances_createdPer90', 'events.fouls_sufferedPer90', 'touches.totalPer90', 'touches.aerial_duel.wonPer90']


Plotting Sunil Chettris Performance Per90 on a radar plot

In [42]:
indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum()

events.goalsPer90                0.931871
events.assistsPer90              0.100615
events.shotsPer90                4.943379
events.shots_on_targetPer90      2.056456
events.chances_createdPer90      1.989116
events.fouls_sufferedPer90       3.072699
touches.totalPer90              78.130668
touches.aerial_duel.wonPer90     1.963568
dtype: float64

In [45]:
#Scaling Values for easier comparison

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
indian_forwards[cols_for_radar] = scaler.fit_transform(indian_forwards[cols_for_radar])
indian_midfielders[cols_for_radar] = scaler.fit_transform(indian_midfielders[cols_for_radar])
indian_defenders[cols_for_radar] = scaler.fit_transform(indian_defenders[cols_for_radar])


In [46]:
indian_forwards[cols_for_radar].max()
indian_midfielders[cols_for_radar].max()
indian_defenders[cols_for_radar].max()

events.goalsPer90               1.0
events.assistsPer90             1.0
events.shotsPer90               1.0
events.shots_on_targetPer90     1.0
events.chances_createdPer90     1.0
events.fouls_sufferedPer90      1.0
touches.totalPer90              1.0
touches.aerial_duel.wonPer90    1.0
dtype: float64

In [47]:
#Plotting Sunil Chettri's Radar Plot as a test
indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum()

events.goalsPer90               0.339098
events.assistsPer90             0.016769
events.shotsPer90               0.164779
events.shots_on_targetPer90     0.137097
events.chances_createdPer90     0.132608
events.fouls_sufferedPer90      0.486511
touches.totalPer90              0.161714
touches.aerial_duel.wonPer90    0.506164
dtype: float64

In [ ]:
#Sunil Chettri's Polar Plot
fig = px.line_polar(indian_forwards, r=indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

Plotting Per90 stats for defenders, midfielders and forwards. Also comparing all the forwards with Sunil Chettri's Polar Plot to analyze the his rival in the ISL

In [ ]:
for i, row in indian_forwards.iterrows():
    print("Index location -->", i, "\n")
    print(row.values, "\n")

In [ ]:
fig = px.line_polar(indian_forwards, r=indian_forwards.loc[(indian_forwards["id"] == row["id"]), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
for i, row in indian_midfielders.iterrows():
    print("Index location -->", i, "\n")
    print(row.values, "\n")

In [ ]:
fig = px.line_polar(indian_midfielders, r=indian_midfielders.loc[(indian_midfielders["id"] == row["id"]), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [ ]:
for i, row in indian_defenders.iterrows():
    print("Index location -->", i, "\n")
    print(row.values, "\n")

In [ ]:
fig = px.line_polar(indian_defenders, r=indian_defenders.loc[(indian_defenders["id"] == row["id"]), cols_for_radar].sum(),
                    theta=cols_for_radar, line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [61]:
ind_fwds_id_names = indian_forwards.drop_duplicates(subset=["id"])[["id", "name"]]
ind_md_id_names = indian_midfielders.drop_duplicates(subset=["id"])[["id", "name"]]
ind_def_id_names = indian_defenders.drop_duplicates(subset=["id"])[["id", "name"]]

In [65]:
isl_max = indian_forwards[cols_for_radar].max().max()
isl_max = indian_midfielders[cols_for_radar].max().max()
isl_max = indian_defenders[cols_for_radar].max().max()

In [63]:
import plotly.graph_objects as go

In [ ]:
#Defenders
for i, row in ind_def_id_names.iterrows():
    print(row["name"])
    # Initiate the plotly go figure
    fig = go.Figure()
    # Add Radar plots for different players:
    fig.add_trace(go.Scatterpolar(
        r=indian_defenders.loc[(indian_defenders["id"] == row["id"]), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name=row["name"]))
    # Additional properties for the plot:
    fig.update_layout(
        title=row["name"],
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, isl_max]
        )),
    showlegend=True
    )
    fig.show()

In [ ]:
#Midfielders
for i, row in ind_md_id_names.iterrows():
    # Initiate the plotly go figure
    fig = go.Figure()
    # Add Radar plots for different players:
    fig.add_trace(go.Scatterpolar(
        r=indian_midfielders.loc[(indian_midfielders["id"] == row["id"]), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name=row["name"]))
    # Additional properties for the plot:
    fig.update_layout(
        title=row["name"],
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, isl_max]
        )),
    showlegend=True
    )
    fig.show()

In [ ]:
#Forwards vs Sunil Chettri
for i, row in ind_fwds_id_names.iterrows():
    if row["id"] == 19150:
        continue
    print(row["name"])
    # Initiate the plotly go figure
    fig = go.Figure()
    # Add Radar plots for different players:
    fig.add_trace(go.Scatterpolar(
        r=indian_forwards.loc[(indian_forwards["id"] == 19150), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name='Sunil Chhetri'))
    fig.add_trace(go.Scatterpolar(
        r=indian_forwards.loc[(indian_forwards["id"] == row["id"]), cols_for_radar].sum(),
        theta=cols_for_radar,
        fill='toself',
        name=row["name"]))
    # Additional properties for the plot:
    fig.update_layout(
        title="Sunil Chhetri vs " + row["name"],
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, isl_max]
        )),
    showlegend=True
    )
    fig.show()

Read.Me

The Indian Soccer League is a young league. With a population of about 1.2 Billion people, Indian tallent in football cannot be overlooked. This project aimes to analyse players from the ISL for seasons 2018 and 2019. The data is extracted from a CSV files that is scraped from the ISL's official website. The project shows the number of minutes each player has played. We also classify the players into 3 catergories: Defenders, Midfielders and Forwards. Further, we calculated the Per90 stat for each player in each position and plotted these statistics on a Polar graph to help understand the weaknesses and strengths of different players in different positions. I also compared each forward with Sunil Chettri's who is the leagues top performer in every statistic to analyze who can be his rival or the next player to reach his level. From looking at the graphs, possible future rivals could be Rahim Ali and Ishan Pandita. 
